# Import Package

In [1]:
# Import Package
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm.notebook import trange
import re
import numpy as np
import seaborn as sns
import json
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import geopandas
import matplotlib.pyplot as plt
import folium
import pandas as pd

# Buat Fungsi Scrapper
Fungsi di bawah ditujukan untuk membuat algoritma web scraping Rumah.com, informasi yang dapat diperoleh dari Rumah.com adalah:
* Nama Rumah
* Lokasi Rumah
* Harga Rumah 
* Luas Bangunan 
* Harga per Area
* Jenis Properti
* Sertifikat Rumah
* Nama Penjual
* Link Referensi di Rumah.com

In [2]:
def house_scrapper(base_url, page = 1):
    # Inisasi Chrome, Jangan lupa spesifikasikan alamat ChromeDriver appnya apabila tidak sesuai
    driver = webdriver.Chrome()
    
    # Buat list kosong
    list_nama_rumah = []
    list_lokasi_rumah = []
    list_harga_rumah = []
    list_kamar_tidur = []
    list_kamar_mandi = []
    list_luas_area = []
    list_luas_per_area = []
    list_jenis_properti = []
    list_sertifikat_rumah = []
    list_tanggal_posting = []
    list_nama_agen = []
    list_referensi = []
    
    # Set base_url
    url = f'{base_url}/{page}?'
        
    # Cari URL rumah, sleep 1 detik untuk load website
    driver.get(url)
    time.sleep(1)
                
    # Cari jumlah rumah yang terjual
    jumlah_rumah = len(driver.find_elements(by = By.XPATH, value = '//*[@id="listings-container"]/div'))
        
    # Loop sampai mencapai record terakhir
    for i in range(1, jumlah_rumah + 1):
        nama_rumah = driver.find_elements(by = By.XPATH, 
                                              value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/div[1]/div/h3/a')
        lokasi_rumah = driver.find_elements(by= By.XPATH, 
                                                value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/div[1]/div/p/span')
        harga_rumah = driver.find_elements(by=By.XPATH, 
                                               value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/ul[1]/li/span[2]')
        kamar_tidur = driver.find_elements(by = By.XPATH, 
                                               value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/ul[2]/li[1]/span[1]')
        kamar_mandi = driver.find_elements(by = By.XPATH, 
                                               value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/ul[2]/li[1]/span[2]')
        luas_area = driver.find_elements(by = By.XPATH, 
                                             value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/ul[2]/li[2]')
        luas_per_area = driver.find_elements(by = By.XPATH,
                                                value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/ul[2]/li[3]')
        jenis_properti = driver.find_elements(by = By.XPATH,
                                                 value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/div[2]/ul/li[1]/span')
        sertifikat_rumah = driver.find_elements(by = By.XPATH,
                                                   value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/div[2]/ul/li[2]/span')
        tanggal_posting = driver.find_elements(by = By.XPATH,
                                                  value = f'//*[@id="listings-container"]/div[{i}]/div[1]/div[2]/div[2]/div/div')
        nama_agen = driver.find_elements(by = By.XPATH,
                                            value = f'//*[@id="listings-container"]/div[{i}]/div[2]/div/div/div[1]/div[2]/div[1]/div/a/span')
        referensi = driver.find_elements(by = By.XPATH,
                                            value = f'//*[@id="listings-container"]/div[{i}]/div[2]/a')

        list_nama_rumah.append(nama_rumah[0].text)
        list_lokasi_rumah.append(lokasi_rumah[0].text)
        list_tanggal_posting.append(tanggal_posting[0].text)
        
        # Informasi opsional yang dapat muncul
        try:
            list_jenis_properti.append(jenis_properti[0].text)
        except:
            list_jenis_properti.append(np.nan)
        try:
            list_sertifikat_rumah.append(sertifikat_rumah[0].text)
        except:
            list_sertifikat_rumah.append(np.nan)
        try:
            list_luas_area.append(luas_area[0].text)
        except:
            list_luas_area.append(np.nan)
        try:
            list_luas_per_area.append(luas_per_area[0].text)
        except:
            list_luas_per_area.append(np.nan)
        try:
            list_harga_rumah.append(harga_rumah[0].text)
        except:
            list_harga_rumah.append(np.nan)
        try:
            list_kamar_tidur.append(kamar_tidur[0].text)
        except:
            list_kamar_tidur.append(np.nan)
        try:
            list_kamar_mandi.append(kamar_mandi[0].text)
        except:
            list_kamar_mandi.append(np.nan)
        
        # Direksi yang berbeda
        try:
            list_nama_agen.append(nama_agen[0].text)
        except:
            nama_agen = driver.find_elements(by = By.XPATH,
                                            value = f'//*[@id="listings-container"]/div[{i}]/div[2]/div/div/div[1]/div/div/div/a/span')
            list_nama_agen.append(nama_agen[0].text)
        list_referensi.append(referensi[0].get_attribute('href'))
    
    # Buat Dataframe
    df = pd.DataFrame({'Nama':list_nama_rumah,
                  'Lokasi': list_lokasi_rumah,
                  'Harga': list_harga_rumah,
                  'Jumlah Kamar Tidur': list_kamar_tidur,
                  'Jumlah Kamar Mandi': list_kamar_mandi,
                  'Total Area': list_luas_area,
                  'Harga per Area': list_luas_per_area,
                  'Jenis Properti': list_jenis_properti,
                  'Sertifikat Rumah': list_sertifikat_rumah,
                  'Tanggal Posting': list_tanggal_posting,
                  'Nama Agen': list_nama_agen,
                  'Link': list_referensi})
    
    # Kembalikan Dataframe
    return df

## WebScraping Rumah di Kota Cimahi
Dari situs Rumah.com terdapat 130 page, untuk melakukan web scraping dilakukan pengulangan sebanyak 130 kali untuk setiap pagenya dikarenakan masalah verifikasi dari Bot Selenium

In [6]:
# Inisiasi start page dan last page scraping
start_page = 1
max_page = 130

# Buat data kosong untuk mengisi informasi hasil scrapping, 
# Jika ada file hasil scrapping sebelumnya, bisa diganti dengan read data Excel sebelumnya lalu lanjut proses WebScraping
df_rumah_cimahi = pd.DataFrame()

# Loop
for page in trange(start_page, max_page + 1):
    # Lakukan WebScraping
    df = house_scrapper(base_url = 'https://www.rumah.com/rumah-dijual/di-area-cimahi-idjb08', 
                        page = page)
    # Modifikasi Tanggal Posting sehingga diperoleh bentuk DateTime
    tanggal_kini = datetime.now()
    df['Tanggal Posting'] = df['Tanggal Posting'].apply(lambda waktu: np.where(waktu.split(' ')[-1] == 'jam', tanggal_kini - timedelta(hours = int(waktu.split(' ')[0])),
                                                                                   np.where(waktu.split(' ')[-1] == 'hari', tanggal_kini - timedelta(days = int(waktu.split(' ')[0])),
                                                                                            np.where(waktu.split(' ')[-1] == 'bulan', tanggal_kini + relativedelta(months=-int(waktu.split(' ')[0])), 
                                                                                                    np.where(waktu.split(' ')[-1] == 'menit', tanggal_kini - timedelta(minutes = int(waktu.split(' ')[0])), 
                                                                                                            np.where(waktu.split(' ')[-1] == 'minggu', tanggal_kini - timedelta(minutes = 7*int(waktu.split(' ')[0])), np.nan))))))
    df['Tanggal Posting'] = df['Tanggal Posting'].apply(lambda dt: dt.strftime('%Y-%m-%d %H:%M:%S.%f'))
    
    # Gabung dengan df_rumah df_rumah
    df_rumah_cimahi = pd.concat([df_rumah_cimahi, df], axis = 0, ignore_index = True)
    
    # Save Data
    df_rumah_cimahi.to_excel('Data_Rumah_Cimahi.xlsx', index = False)

  0%|          | 0/47 [00:00<?, ?it/s]

C:\Users\Asus\AppData\Local\Temp\ipykernel_44028\1880212073.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\Asus\Documents\Web Scraping\chromedriver-win64\chromedriver.exe")

KeyboardInterrupt



## WebScraping Rumah di Kota Bandung
Dari Rumah.com per 7 Agustus 2023, terdapat 2791 pages yang menyediakan informasi harga rumah di kota Bandung. Pada kasus ini hanya dilakukan web scraping sebanyak 2400 pages dengan estimasi waktu scraping 400 pages adalah sekitar 3 jam

In [3]:
# Inisiasi start page dan last page scraping
start_page = 1
max_page = 400

# Buat data kosong untuk mengisi informasi hasil scrapping, 
# Jika ada file hasil scrapping sebelumnya, bisa diganti dengan read data Excel sebelumnya lalu lanjut proses WebScraping
df_rumah_bandung = pd.DataFrame()

# Loop
for page in trange(start_page, max_page + 1):
    # Lakukan proses webscraping
    df = house_scrapper(base_url = 'https://www.rumah.com/rumah-dijual/di-area-bandung-idjb01',
                        page = page)
    
    # Sesuaikan tanggal posting dengan mengubah formatnya menjadi format tanggal berdasarkan waktu scraping berlangsung
    tanggal_kini = datetime.now()
    df['Tanggal Posting'] = df['Tanggal Posting'].apply(lambda waktu: np.where(waktu.split(' ')[-1] == 'jam', tanggal_kini - timedelta(hours = int(waktu.split(' ')[0])),
                                                                                   np.where(waktu.split(' ')[-1] == 'hari', tanggal_kini - timedelta(days = int(waktu.split(' ')[0])),
                                                                                            np.where(waktu.split(' ')[-1] == 'bulan', tanggal_kini + relativedelta(months=-int(waktu.split(' ')[0])), 
                                                                                                    np.where(waktu.split(' ')[-1] == 'menit', tanggal_kini - timedelta(minutes = int(waktu.split(' ')[0])), 
                                                                                                            np.where(waktu.split(' ')[-1] == 'minggu', tanggal_kini - timedelta(minutes = 7*int(waktu.split(' ')[0])), np.nan))))))
    df['Tanggal Posting'] = df['Tanggal Posting'].apply(lambda dt: dt.strftime('%Y-%m-%d %H:%M:%S.%f'))
    
    # Gabung data scrapping sebelumnya dengan data hasil scrapping sekarang
    df_rumah_bandung = pd.concat([df_rumah_bandung, df], axis = 0, ignore_index = True)
    
    # Save File
    df_rumah_bandung.to_excel('Data_Rumah_Bandung.xlsx', index = False)

  0%|          | 0/400 [00:00<?, ?it/s]

# Buat Fungsi Pembersihan Data
Untuk membersihkan data hasil scrapping, gunakan fungsi clean_data_cimahi atau clean_data_bandung untuk data hasil scrapping Kota Cimahi dan Bandung berturut-turut

In [4]:
def clean_data_cimahi(data):
    # Cleaning Lokasi
    data['Provinsi'] = data['Lokasi'].apply(lambda alamat: 'JAWA BARAT' if re.search('JAWA BARAT', alamat.upper()) else 'LAINNYA')
    data['Kota'] = data['Lokasi'].apply(lambda alamat: 'CIMAHI' if re.search('CIMAHI', alamat.upper()) else 'LAINNYA')
    data['Kecamatan'] = data['Lokasi'].apply(lambda alamat: np.where(re.search('CIMAHI UTARA', alamat.upper()), 'CIMAHI UTARA',
                                                                    np.where(re.search('CIMAHI TENGAH', alamat.upper()), 'CIMAHI TENGAH',
                                                                            np.where(re.search('CIMAHI SELATAN', alamat.upper()), 'CIMAHI SELATAN', 'LAINNYA'))))
    
    data['Jalan'] = data['Lokasi'].apply(lambda alamat: alamat.split(',')[0].strip().upper())
    
    # Cleaning Harga
    data['Harga'] = data['Harga'].fillna('0 jt')
    data['Harga'] = data['Harga'].apply(lambda harga: harga.replace(',','.'))
    data['Harga'] = data['Harga'].apply(lambda harga: float(harga.split(' ')[0]) * 10**9 if harga.split(' ')[-1] == 'M' else float(harga.split(' ')[0]) * 10**6)
    data['Harga'] = data['Harga'].replace(0.0, data['Harga'].median())
    
    # Cleaning Jumlah Kamar Tidur
    data['Jumlah Kamar Tidur'] = data['Jumlah Kamar Tidur'].replace(lambda val: np.nan if val.isnumeric() == False else val)
    
    # Cleaning Total Area
    data['Total Area'] = data['Total Area'].fillna('0 m²')
    data['Total Area'] = data['Total Area'].apply(lambda area: float(area.replace('m²','').strip()))
    data['Total Area'] = data['Total Area'].replace(0.0, data['Total Area'].median())
    
    # Cleaning Harga per Area
    data['Harga per Area'] = data['Harga per Area'].fillna('0 m²')
    data['Harga per Area'] = data['Harga per Area'].apply(lambda area: area.replace('m²','').strip())
    data['Harga per Area'] = data['Harga per Area'].apply(lambda harga: ''.join([c for c in harga if c.isnumeric()]))
    data['Harga per Area'] = data['Harga per Area'].replace(0.0, data['Harga per Area'].median())
    
    # Return df
    return data

In [5]:
def clean_data_bandung(data):
    # Cleaning Lokasi
    data['Provinsi'] = data['Lokasi'].apply(lambda alamat: 'JAWA BARAT' if re.search('JAWA BARAT', alamat.upper()) else 'LAINNYA')
    data['Kota'] = data['Lokasi'].apply(lambda alamat: 'BANDUNG' if re.search('BANDUNG', alamat.upper()) else 'LAINNYA')
    data['Kecamatan'] = data['Lokasi'].apply(lambda alamat: alamat.split(',')[-3].strip().upper())
    
    # Buat dictionary kecamatan untuk memetakan nilai kecamatan yang salah ke yang benar
    kecamatan = {'DAGO':'COBLONG', 'KOTA BARU PARAHYANGAN':'PADALARANG', 'SOEKARNO HATTA':'BUAHBATU', 'UJUNG BERUNG':'UJUNGBERUNG', 'GEGER KALONG':'SUKASARI',
                'CIGADUNG':'CIBEUNYING KALER', 'TURANGGA':'LENGKONG', 'ANCOL':'REGOL', 'LEMBANG CIBODAS':'LEMBANG', 'ASIA AFRIKA':'SUMUR BANDUNG',
                'PASTEUR':'SUKAJADI', 'SETIABUDI':'SUKASARI', 'MEKAR WANGI':'BOJONGLOA KIDUL', 'SETRA DUTA':'SUKASARI', 'SETRASARI':'SUKAJADI',
                'GARUDA':'ANDIR', 'BURANGRANG':'LENGKONG', 'GUNUNG BATU':'CILEDUG','SUDIRMAN':'ANDIR',
                'TAMAN HOLLIS INDAH':'BANDUNG KULON', 'KOPO':'BOJONGLOA KALER', 'TAMAN KOPO INDAH':'BOJONGLOA KALER',
                'CIWASTRA':'RANCASARI','RIAU RE MARTADINATA':'BANDUNG WETAN', 'CIBADUYUT':'BOJONGLOA KIDUL', 
                'SINGGASANA PRADANA':'BOJONGLOA KIDUL', 'PASIR LUYU':'REGOL', 'CIUMBULEUIT':'CIDADAP', 
                'MARGACINTA':'BUAHBATU', 'SURYA SUMANTRI':'SUKAJADI', 'SARIJADI':'SUKASARI', 'PINUS REGENSI':'ARCAMANIK',
                'BKR KEMBAR':'LENGKONG','SETRA MURNI':'SUKASARI', 'PAJAJARAN':'CICENDO','KOPO PERMAI':'BOJONGLOA KALER',
                'CIWEDEY':'CIWIDEY','PASIR KOJA':'BOJONGLOA KIDUL','DR. CURIE CIPAGANTI':'CICENDO','CIATEUL':'REGOL',
                'GATOT SUBROTO':'LENGKONG','SETRA INDAH':'SUKAJADI','H. JUANDA DAGO':'COBLONG','SRIMAHI':'REGOL',
                'KARAPITAN':'LENGKONG','CIMINDI':'ANDIR', 'MOCH.RAMDHAN':'REGOL','LASWI LINGKAR SELATAN':'BATUNUNGGAL',
                'OTISTA':'ASTANAANYAR','CIJERAH':'BANDUNG KULON' ,'SOREWANG':'SOREANG'}
    
    # Replace Kecamatan
    data['Kecamatan'] = data['Kecamatan'].replace(kecamatan)
    
    data['Jalan'] = data['Lokasi'].apply(lambda alamat: alamat.split(',')[0].strip().upper())
    
    # Cleaning Harga
    data['Harga'] = data['Harga'].fillna('0 jt')
    data['Harga'] = data['Harga'].apply(lambda harga: harga.replace('.','') if type(harga) == str else harga)
    data['Harga'] = data['Harga'].apply(lambda harga: harga.replace(',','.') if type(harga) == str else harga)
    data['Harga'] = data['Harga'].apply(lambda harga: np.where(harga.split(' ')[-1] == 'M', float(harga.split(' ')[0]) * 10**9,
                                                              np.where(harga.split(' ')[-1] == 'jt', float(harga.split(' ')[0]) * 10**6, 
                                                                      np.where(harga.split(' ')[-1] == 'rb', float(harga.split(' ')[0]) * 10**3, harga))))
    data['Harga'] = data['Harga'].astype('float64')
    data['Harga'] = data['Harga'].replace(0, data['Harga'].median())

    # Cleaning Jumlah Kamar Tidur
    data['Jumlah Kamar Tidur'] = data['Jumlah Kamar Tidur'].replace(lambda val: np.nan if val.isnumeric() == False else val)
    
    # Total Area
    data['Total Area'] = data['Total Area'].fillna('0 m²')
    data['Total Area'] = data['Total Area'].apply(lambda area: int(area.replace('m²','').strip()) if type(area) == str else area)
    data['Total Area'] = data['Total Area'].astype('int64')
    data['Total Area'] = data['Total Area'].replace(0.0, data['Total Area'].median())
    
    # Harga per Area
    data['Harga per Area'] = data['Harga per Area'].fillna('0 m²')
    data['Harga per Area'] = data['Harga per Area'].apply(lambda area: area.replace('m²','').strip() if type(area) == str else area)
    data['Harga per Area'] = data['Harga per Area'].apply(lambda area: ''.join([c for c in area if c.isnumeric()]))
    data['Harga per Area'] = data['Harga per Area'].astype('int64')
    data['Harga per Area'] = data['Harga per Area'].replace(0, data['Harga per Area'].median())
    
    # Return df
    return data

## Pembersihan Data Kota Cimahi

In [ ]:
# Buat Copy untuk jaga-jaga
df_copy_cimahi = df_rumah_cimahi.copy()

# Lakukan pembersihan data
df_cimahi_cleaned = clean_data_cimahi(df_copy_cimahi)

In [ ]:
# Read Data Hasil Scrapping
df_cimahi_prev = pd.read_excel('Data_Rumah_Cimahi_Cleaned.xlsx')

# Gabung dengan data hasil scrapping sebelumnya
df_cimahi_cleaned = pd.concat([df_cimahi_ori, df_cimahi_cleaned], axis = 0, ignore_index = True)

# Save data
df_cimahi_cleaned_sub.to_excel('Data_Rumah_Cimahi_Cleaned.xlsx', index = False)

## Pembersihan Data Kota Bandung

In [11]:
# Buat Copy untuk jaga-jaga
df_copy_bandung = df_rumah_bandung.copy()

# Lakukan pembersihan data
df_bandung_cleaned = clean_data_bandung(df_copy)

In [13]:
# Read Data Hasil Scrapping
df_bandung_prev = pd.read_excel('Data_Rumah_Bandung_Cleaned.xlsx')

# Gabung dengan data hasil scrapping sebelumnya
df_bandung_cleaned = pd.concat([df_bandung_ori, df_bandung_cleaned], axis = 0, ignore_index = True)
df_bandung_cleaned

,Nama,Lokasi,Harga,Jumlah Kamar Tidur,Jumlah Kamar Mandi,Total Area,Harga per Area,Jenis Properti,Sertifikat Rumah,Tanggal Posting,Nama Agen,Link,Provinsi,Kota,Kecamatan,Jalan
0,Valle Verde,"33 Jl. Pasirhalang, Andir, Bandung, Jawa Barat",1.846108e+09,2,2.0,98,12612613,Rumah,SHM - Sertifikat Hak Milik,2023-08-02 19:05:21.187276,PT Anugerah Cipta Asri,https://www.rumah.com/listing-properti/proyek/...,JAWA BARAT,BANDUNG,ANDIR,33 JL. PASIRHALANG
1,Dijual Cepat Rumah Mewah American Style Infini...,"Golf Island Bukit Pakar Golf Dago Resort, Dago...",1.150000e+10,4,4.0,500,21739130,Rumah,SHM - Sertifikat Hak Milik,2023-07-03 07:05:21.187276,Gin Ginanjar,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,COBLONG,GOLF ISLAND BUKIT PAKAR GOLF DAGO RESORT
2,Kota Baru Parahyangan,"6 Jalan Panyawangan Kav 6-B, Kota Baru Parahya...",8.000000e+08,3,3.0,75,9523810,Rumah,SHM - Sertifikat Hak Milik,2023-08-03 06:58:21.187276,Hazelia Umaiiroh,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,PADALARANG,6 JALAN PANYAWANGAN KAV 6-B
3,Dijual Cepat Rumah Kost di Tubagus Ismail Saya...,"Tubagus Ismail-Dago Bandung, Dago, Bandung, Ja...",6.750000e+09,10,10.0,650,10957792,Rumah,SHM - Sertifikat Hak Milik,2023-07-03 07:05:21.187276,Gin Ginanjar,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,COBLONG,TUBAGUS ISMAIL-DAGO BANDUNG
4,Kota Baru Parahyangan,"6 Jalan Panyawangan Kav 6-B, Kota Baru Parahya...",5.489549e+09,5,5.0,265,18929481,Rumah,SHGB - Hak Guna Bangunan,2023-08-03 06:10:21.187276,Hendra .,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,PADALARANG,6 JALAN PANYAWANGAN KAV 6-B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45200,"Rumah Baru, Harga Promo !! di Sindanglaya, Arc...","Arcamanik, Bandung, Jawa Barat",4.940000e+08,3,2,72,8233333,Rumah,SHM - Sertifikat Hak Milik,2023-06-08 11:28:19.249957,Andy Halim,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,ARCAMANIK,ARCAMANIK
45201,"Rumah siap huni dan terawat di Area Turangga, ...","Turangga, Bandung, Jawa Barat",1.950000e+09,4,3,220,14885496,Rumah,SHM - Sertifikat Hak Milik,2023-06-08 11:28:19.249957,Andy Halim,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,LENGKONG,TURANGGA
45202,"Rumah Terawat dan Siap huni, sudah renovasi ar...","Turangga, Bandung, Jawa Barat",1.950000e+09,4,3,170,18571429,Rumah,SHM - Sertifikat Hak Milik,2023-06-08 11:28:19.249957,Andy Halim,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,LENGKONG,TURANGGA
45203,Rumah terawat dan siap huni sudah renovasi are...,"Turangga, Bandung, Jawa Barat",2.200000e+09,4,3,180,18803419,Rumah,SHM - Sertifikat Hak Milik,2023-06-08 11:28:19.249957,Andy Halim,https://www.rumah.com/listing-properti/dijual-...,JAWA BARAT,BANDUNG,LENGKONG,TURANGGA


In [16]:
# Filter Bandung Utara, Selatan, Tengah, Timur
bandung_utara = df_bandung_cleaned[df_bandung_cleaned['Kecamatan'] == 'BANDUNG UTARA']
bandung_selatan = df_bandung_cleaned[df_bandung_cleaned['Kecamatan'] == 'BANDUNG SELATAN']
bandung_tengah = df_bandung_cleaned[df_bandung_cleaned['Kecamatan'] == 'BANDUNG TENGAH']
bandung_timur = df_bandung_cleaned[df_bandung_cleaned['Kecamatan'] == 'BANDUNG TIMUR']


# Print jumlah observasi di Bandung Utara, Selatan, Tengah, Timur
total_observasi = bandung_utara.shape[0] + bandung_selatan.shape[0] + bandung_tengah.shape[0] + bandung_timur.shape[0]
print('Jumlah Observasi:', total_observasi)
print(f'Rasio Observasi: {total_observasi*100/df_bandung_cleaned.shape[0]} %')

Jumlah Observasi: 605
Rasio Observasi: 1.3383475279283266 %


Karena hanya 1% observasi yang terletak di daerah kecamatan Bandung Utara, Selatan, Tengah, Timur, padahal dalam database tidak ada nama 4 kecamatan tersebut, maka observasi di 4 wilayah tersebut didrop

In [17]:
# Buat list berisi bandung utara, selatan, tengah, timur
kecamatan_tidak_diketahui = ['BANDUNG UTARA','BANDUNG SELATAN','BANDUNG TENGAH', 'BANDUNG TIMUR']

# Filter semua kecamatan kecuali yang tidak diketahui di atas
df_bandung_cleaned = df_bandung_cleaned[~df_bandung_cleaned['Kecamatan'].isin(kecamatan_tidak_diketahui)].reset_index(drop = True)

Diketahui dari data hasil scrapping ada harga rumah yang tidak masuk akal seperti ratusan ribu rupiah, maka dari itu filter harga rumah yang lebih dari 1 juta rupiah

In [18]:
# Discard Harga Rumah di bawah 1 jt
df_bandung_cleaned_sub = df_bandung_cleaned[df_bandung_cleaned['Harga'] > 10**6].reset_index(drop = True)

In [19]:
df_bandung_cleaned_sub.to_excel('Data_Rumah_Bandung_Cleaned.xlsx', index = False)